In [ ]:
# we use Pandas to work with the data as it makes working with categorical data very easy
import pandas as pd
from google.colab import files

In [114]:
# this is a list of the column names in our dataset (as the file doesn't contain any headers)
names = (
    'Data',
    'Idade',
    'Doenca',
)

df = pd.read_csv("alunos.csv", sep=";", header=None, names=names, index_col=False, engine='python');# We load the data using Pandas
df = df.iloc[1: , :]

In [ ]:
def idade(nivel):
  if(nivel == 1):
    df['Idade'] = df['Data'].dt.strftime('%d-%m-%Y')
  elif(nivel == 2):
    df['Idade']= df['Idade'].replace(['16','17', '18', '19', '20'],['[15-17]','[15-17]', '[18-20]', '[18-20]', '[18-20]'])
  elif(nivel == 3):
      df['Idade'] = df['Idade'].apply(lambda x: "[16-20]")


      

In [113]:
# df.loc[df['Idade'] == 18 or df['Idade'] == 17, 'Idade'] = '15-17'

def data(nivel):
  df['Data'] = pd.to_datetime(df.Data, errors = 'coerce')
  if(nivel == 1):
      df['Data'] = df['Data'].dt.strftime('%d-%m-%Y')
  elif(nivel == 2):
      df['Data'] = df['Data'].dt.strftime('%m-%y')
  elif(nivel == 3):
      df['Data'] = df['Data'].dt.strftime('%Y')
  elif(nivel == 4): # decada
    df['Data'] = df['Data'].apply(lambda x: (x.year)*10)  # AAAAAAAAAAAAAAAAAAAAAAAAAAAAA


In [ ]:
i = 1
for i in df['Data']:
  j = i + 1
  for j in df['Data']:
    if (df['Data'][i] == df['Data'][j]):
      print(df['Data'],"linha ", df['Data'][i], df['Data'][j])
    j = j + 1
  i = i+1

In [115]:
df = df.groupby(by=['Idade','Data'])['Doenca'].apply(lambda x: x.sum()).reset_index() # create partitions 

In [116]:
def is_k_anonymous(df, partition, sensitive_column, k=3):
    """
    :param               df: The dataframe on which to check the partition.
    :param        partition: The partition of the dataframe to check.
    :param sensitive_column: The name of the sensitive column
    :param                k: The desired k
    :returns               : True if the partition is valid according to our k-anonymity criteria, False otherwise.
    """
    if len(partition) < k:
        return False
    return True

def partition_dataset(df, feature_columns, sensitive_column, scale, is_valid):
    """
    :param               df: The dataframe to be partitioned.
    :param  feature_columns: A list of column names along which to partition the dataset.
    :param sensitive_column: The name of the sensitive column (to be passed on to the `is_valid` function)
    :param            scale: The column spans as generated before.
    :param         is_valid: A function that takes a dataframe and a partition and returns True if the partition is valid.
    :returns               : A list of valid partitions that cover the entire dataframe.
    """
    finished_partitions = []
    partitions = [df.index]
    while partitions:
        partition = partitions.pop(0)
        spans = get_spans(df[feature_columns], partition, scale)
        for column, span in sorted(spans.items(), key=lambda x:-x[1]):
            lp, rp = split(df, partition, column)
            if not is_valid(df, lp, sensitive_column) or not is_valid(df, rp, sensitive_column):
                continue
            partitions.extend((lp, rp))
            break
        else:
            finished_partitions.append(partition)
    return finished_partitions


,Idade,Data,Doenca
0,16,1/5/2009,meningite
1,16,1/6/2014,catapora
2,16,1/7/2011,meningite
3,16,1/7/2017,sarampo
4,16,10/2/2019,gripe
...,...,...,...
294,20,8/1/2011,meningite
295,20,8/6/2008,catapora
296,20,8/6/2017,sarampo
297,20,9/3/2017,sarampo


In [117]:
data(2)
# df['Idade'] = df['Idade'].astype(int)

#idade(2)

In [118]:
df

,Idade,Data,Doenca
0,16,01-09,meningite
1,16,01-14,catapora
2,16,01-11,meningite
3,16,01-17,sarampo
4,16,10-19,gripe
...,...,...,...
294,20,08-11,meningite
295,20,08-08,catapora
296,20,08-17,sarampo
297,20,09-17,sarampo


In [ ]:
# df.to_csv('file_name.csv', index=False)
# files.download('file_name.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>